Notebook to use the helper_CI tools to build the QED-CIS matrix using
$$ <ij||kl> + D_{ij||kl} + T + V + Q + d $$

Note we might want to look at these equations [here](https://www.chem.fsu.edu/~deprince/programming_projects/cis/).
Ideally, this will yield the same result as ../simple_cis.ipynb.

In [1]:
import time
import numpy as np
np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_cqed_rhf import cqed_rhf


In [2]:
def spin_idx_to_spat_idx_and_spin(P):
    """ function to take the numeric label of a spin orbital
        and return the spatial index and the spin index separately.
        Starts counting from 0:
        
        Arguments
        ---------
        P : int
            spin orbital label
        
        Returns
        -------
        [p, spin] : numpy array of ints
            p is the spatial orbital index and spin is the spin index.
            spin = 1  -> alpha
            spin = -1 -> beta
            
        Example
        -------
        >>> spin_idx_to_spat_idx_and_spin(0)
        >>> [0, 1]
        >>> spin_idx_to_spat_idx_and_spin(3)
        >>> [1, -1]
        
    """
    spin = 1
    if P % 2 == 0:
        p = P / 2
        spin = 1
    else:
        p = (P-1) / 2
        spin = -1
    return np.array([p, spin], dtype=int)


def map_spatial_to_spin(tei_spatial, I, J, K, L):
    """ function to take two electron integrals in the spatial orbital basis
        in chemist notation along with 4 indices I, J, K, L and return
        the corresponding two electron integral in the spin orbital basis
        in phycisit notation, <IJ||KL>
    
    """
    # Phys to Chem: <IJ||KL> -> [IK|JL] - [IL|JK]
    i_s = spin_idx_to_spat_idx_and_spin(I)
    k_s = spin_idx_to_spat_idx_and_spin(K)
    j_s = spin_idx_to_spat_idx_and_spin(J)
    l_s = spin_idx_to_spat_idx_and_spin(L)
    
    #print(i_s[1])
    # (ik|jl)
    spat_ikjl = tei_spatial[i_s[0], k_s[0], j_s[0], l_s[0]] * ( i_s[1] == k_s[1] ) *  ( j_s[1] == l_s[1] )
    
    # (il|jk)
    spat_iljk = tei_spatial[i_s[0], l_s[0], j_s[0], k_s[0]] * ( i_s[1] == l_s[1] ) *  ( j_s[1] == k_s[1] )
    
    return spat_ikjl - spat_iljk

def map_spatial_dipole_to_spin(mu, I, J, K, L):
    """ function to take two electron integrals in the spatial orbital basis
        in chemist notation along with 4 indices I, J, K, L and return
        the corresponding two electron integral in the spin orbital basis
        in phycisit notation, <IJ||KL>
    
    """
    # Phys to Chem: <IJ||KL> -> [IK|JL] - [IL|JK]
    i_s = spin_idx_to_spat_idx_and_spin(I)
    k_s = spin_idx_to_spat_idx_and_spin(K)
    j_s = spin_idx_to_spat_idx_and_spin(J)
    l_s = spin_idx_to_spat_idx_and_spin(L)
    
    #print(i_s[1])
    # (ik|jl)
    spat_ikjl = mu[i_s[0], k_s[0]] * mu[j_s[0], l_s[0]] * ( i_s[1] == k_s[1] ) *  ( j_s[1] == l_s[1] )
    
    # (il|jk)
    spat_iljk = mu[i_s[0], l_s[0]] * mu[j_s[0], k_s[0]] * ( i_s[1] == l_s[1] ) *  ( j_s[1] == k_s[1] )
    
    return spat_ikjl - spat_iljk

In [3]:
# some test options
neglect_mumu = False
neglect_oe_terms = False

In [12]:

# Check energy against psi4?
compare_psi4 = True

# Memory for Psi4 in GB
# psi4.core.set_memory(int(2e9), False)
psi4.core.set_output_file('output.dat', False)

# Memory for numpy in GB
numpy_memory = 2

mol_str = """
H 
H 1 1.0
symmetry c1
"""

options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8
                  }



# photon energy
omega_val = 4.75 / psi4.constants.Hartree_energy_in_eV

# define the lambda vector
lambda_vector = np.array([0.05, 0.05, 0.05])

mol = psi4.geometry(mol_str)


psi4.set_options(options_dict)

print('\nStarting SCF and integral build...')
t = time.time()

# First compute SCF energy using Psi4
scf_e, wfn = psi4.energy('SCF', return_wfn=True)

# now compute cqed-rhf to get transformation vectors with cavity
cqed_rhf_dict = cqed_rhf(lambda_vector, mol_str, options_dict)

# grab necessary quantities from cqed_rhf_dict
scf_e = cqed_rhf_dict["RHF ENERGY"]
cqed_scf_e = cqed_rhf_dict["CQED-RHF ENERGY"]
wfn = cqed_rhf_dict["PSI4 WFN"]
C = cqed_rhf_dict["CQED-RHF C"]
D = cqed_rhf_dict["CQED-RHF DENSITY MATRIX"]
eps = cqed_rhf_dict["CQED-RHF EPS"]
dc = cqed_rhf_dict["DIPOLE ENERGY"]

# collect rhf wfn object as dictionary
wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# update wfn_dict with orbitals from CQED-RHF
wfn_dict["matrix"]["Ca"] = C
wfn_dict["matrix"]["Cb"] = C
# update wfn object
wfn = psi4.core.Wavefunction.from_file(wfn_dict)

# Grab data from wavfunction class
Ca = wfn.Ca()
ndocc = wfn.doccpi()[0]
nmo = wfn.nmo()

# Compute size of Hamiltonian in GB
from scipy.special import comb
nDet = comb(nmo, ndocc)**2
H_Size = nDet**2 * 8e-9
print('\nSize of the Hamiltonian Matrix will be %4.2f GB.' % H_Size)
if H_Size > numpy_memory:
    #clean()
    raise Exception("Estimated memory utilization (%4.2f GB) exceeds numpy_memory \
                    limit of %4.2f GB." % (H_Size, numpy_memory))

# Integral generation from Psi4's MintsHelper
t = time.time()
mints = psi4.core.MintsHelper(wfn.basisset())



Starting SCF and integral build...

Start SCF iterations:

Canonical RHF One-electron energy = -2.2216883593010519
CQED-RHF One-electron energy      = -2.2148046877305525
Nuclear repulsion energy          = 0.5291772106700000
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -1.0592249776148162   dE = -1.05922E+00   dRMS = 1.40570E-16
SCF Iteration   2: Energy = -1.0592249776148157   dE =  4.44089E-16   dRMS = 3.69647E-17
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -1.05922498 hartree
Psi4  SCF energy: -1.06610865 hartree

Size of the Hamiltonian Matrix will be 0.00 GB.


In [5]:
# dipole arrays in AO basis
mu_ao_x = np.asarray(mints.ao_dipole()[0])
mu_ao_y = np.asarray(mints.ao_dipole()[1])
mu_ao_z = np.asarray(mints.ao_dipole()[2])

# transform dipole array to HF-PF basis
mu_cmo_x = np.dot(C.T, mu_ao_x).dot(C)
mu_cmo_y = np.dot(C.T, mu_ao_y).dot(C)
mu_cmo_z = np.dot(C.T, mu_ao_z).dot(C)

# electronic dipole expectation value with CQED-RHF density
mu_exp_x = np.einsum("pq,pq->", 2 * mu_ao_x, D)
mu_exp_y = np.einsum("pq,pq->", 2 * mu_ao_y, D)
mu_exp_z = np.einsum("pq,pq->", 2 * mu_ao_z, D)

# get electronic dipole expectation value
mu_exp_el = np.array([mu_exp_x, mu_exp_y, mu_exp_z])

# \lambda \cdot < \mu > where < \mu > contains only electronic terms 
l_dot_mu_exp = np.dot(lambda_vector, mu_exp_el)

# \lambda \cdot \mu_{el} in ao basis
l_dot_mu_el = lambda_vector[0] * mu_ao_x
l_dot_mu_el += lambda_vector[1] * mu_ao_y
l_dot_mu_el += lambda_vector[2] * mu_ao_z

# \lambda \cdot \mu_{el} in PF basis basis
l_dot_mu_el_cmo = lambda_vector[0] * mu_cmo_x
l_dot_mu_el_cmo += lambda_vector[1] * mu_cmo_y
l_dot_mu_el_cmo += lambda_vector[2] * mu_cmo_z

# get \lambda * <\mu>_e \lambda * \hat{\mu} term
d_PF = -l_dot_mu_exp * l_dot_mu_el

# quadrupole arrays
Q_ao_xx = np.asarray(mints.ao_quadrupole()[0])
Q_ao_xy = np.asarray(mints.ao_quadrupole()[1])
Q_ao_xz = np.asarray(mints.ao_quadrupole()[2])
Q_ao_yy = np.asarray(mints.ao_quadrupole()[3])
Q_ao_yz = np.asarray(mints.ao_quadrupole()[4])
Q_ao_zz = np.asarray(mints.ao_quadrupole()[5])


# Pauli-Fierz 1-e quadrupole terms, Line 2 of Eq. (9) in [McTague:2021:ChemRxiv]
Q_PF = -0.5 * lambda_vector[0] * lambda_vector[0] * Q_ao_xx
Q_PF -= 0.5 * lambda_vector[1] * lambda_vector[1] * Q_ao_yy
Q_PF -= 0.5 * lambda_vector[2] * lambda_vector[2] * Q_ao_zz

# accounting for the fact that Q_ij = Q_ji
# by weighting Q_ij x 2 which cancels factor of 1/2
Q_PF -= lambda_vector[0] * lambda_vector[1] * Q_ao_xy
Q_PF -= lambda_vector[0] * lambda_vector[2] * Q_ao_xz
Q_PF -= lambda_vector[1] * lambda_vector[2] * Q_ao_yz



In [13]:
# preparing 1e- and 2e- arrays
H = np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential()) + Q_PF + d_PF



print('\nTotal time taken for ERI integrals: %.3f seconds.\n' % (time.time() - t))

#Make spin-orbital MO
print('Starting AO -> spin-orbital MO transformation...')
t = time.time()
MO = np.asarray(mints.mo_spin_eri(Ca, Ca))

nso = 2 * nmo
TDI_spin = np.zeros((nso, nso, nso, nso))
for i in range(nso):
    for j in range(nso):
        for k in range(nso):
            for l in range(nso):
                TDI_spin[i, j, k, l] = map_spatial_dipole_to_spin(l_dot_mu_el_cmo, i, j, k, l)
                
# check to see if we should ignore dipole-dipole terms
if neglect_mumu:
    print("neglecting dipole dipole")
else:
    MO += TDI_spin

# Update H, transform to MO basis and tile for alpha/beta spin
H = np.einsum('uj,vi,uv', Ca, Ca, H)
H = np.repeat(H, 2, axis=0)
H = np.repeat(H, 2, axis=1)

# prepare the g matrix by transforming the l_dot_mu_el matrix
g_mat = np.einsum('uj,vi,uv', Ca, Ca, l_dot_mu_el)
g_mat = np.repeat(g_mat, 2, axis=0)
g_mat = np.repeat(g_mat, 2, axis=1)


#print(H)

# Make H block diagonal
spin_ind = np.arange(H.shape[0], dtype=np.int) % 2
H *= (spin_ind.reshape(-1, 1) == spin_ind)
g_mat *= (spin_ind.reshape(-1, 1) == spin_ind)
g_mat *= -np.sqrt(omega_val/2)

print('..finished transformation in %.3f seconds.\n' % (time.time() - t))






Total time taken for ERI integrals: 733.138 seconds.

Starting AO -> spin-orbital MO transformation...
..finished transformation in 0.005 seconds.



/var/folders/5f/ftf5xlgj4dv_6xxqh_p9pwyc0000gq/T/ipykernel_73644/1785954010.py:41: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  spin_ind = np.arange(H.shape[0], dtype=np.int) % 2


Here we are going to generate the set of all singly-excited determinants $|\Phi_i^a\rangle$
that we will call `singlesDets` and the reference determinant $|\Phi_0\rangle$ 
that we will calle `refDet`.

In [10]:
# prepare the determinant list
from helper_CI import Determinant, HamiltonianGenerator, compute_excitation_level
from itertools import combinations

print('Generating singly-excited determinants')
t = time.time()
singlesDets = []
for alpha in combinations(range(nmo), ndocc):
    alpha_ex_level = compute_excitation_level(alpha, ndocc)
    for beta in combinations(range(nmo), ndocc):
        beta_ex_level = compute_excitation_level(beta, ndocc)
        if alpha_ex_level + beta_ex_level == 1:
            print(F' adding alpha: {alpha} and beta: {beta}\n') 
            singlesDets.append(Determinant(alphaObtList=alpha, betaObtList=beta))

print('..finished generating singly-excited determinants in %.3f seconds.\n' % (time.time() - t))
print(F'..there are {len(singlesDets)} determinants \n')
for i in range(len(singlesDets)):
    print(singlesDets[i])
    
print('Generating reference determinant')

occList = [i for i in range(ndocc)]
refDet = Determinant(alphaObtList=occList, betaObtList=occList)
print(refDet)


Generating singly-excited determinants
 adding alpha: (0,) and beta: (1,)

 adding alpha: (1,) and beta: (0,)

..finished generating singly-excited determinants in 0.000 seconds.

..there are 2 determinants 

|[0][1]>
|[1][0]>
Generating reference determinant
|[0][0]>


##### this illustrates some functionality of the determinant class.
numDet = len(detList)
print(numDet)
for i in range(numDet):
    for j in range(numDet):
        unique1, unique2, sign = detList[i].getUniqueOrbitalsInMixIndexListsPlusSign(detList[j])
        print(detList[i], detList[j], unique1, unique2, sign)
        


First we are going to generate the ${\bf A + \Delta} + n\hbar{\bf \Omega}$ block of the Hamiltonian
which results from $\langle \Phi_i^a,n | \hat{H} | \Phi_j^b,n\rangle$ terms.
We can compute ${\bf A + \Delta}$ once and then add it to the appropriate block of
the Hamiltonian matrix with the appropriate block of $n\hbar{\bf \Omega}$. 

In [11]:
print('Generating A+\Delta sublock...')
A_D_generator = HamiltonianGenerator(H, MO)


A_D_matrix = A_D_generator.generateMatrix(singlesDets)
print("printing H")
print(A_D_matrix)

Generating A+\Delta sublock...
printing H
[[-1.07094  0.19975]
 [ 0.19975 -1.07094]]


Next we are going to generate the ${\bf g}$ blocks which arise from $ -\sqrt{\frac{\omega}{2}}  \langle \Phi_i^a,n | {\bf \lambda} \cdot {\bf \hat{\mu}} (\hat{b}^{\dagger} + \hat{b}) | \Phi_j^b,m\rangle$

In [18]:
print('Generating g sublock...')
g_generator = HamiltonianGenerator(g_mat, MO * 0)

G_matrix = g_generator.generateMatrix(singlesDets)
print("printing g")
print(G_matrix)

Generating g sublock...
printing g
[[0. 0.]
 [0. 0.]]


Next we are going to generate the ${\bf G}$ blocks which arise from $ -\sqrt{\frac{\omega}{2}}  \langle \Phi_0,n | {\bf \lambda} \cdot {\bf \hat{\mu}} (\hat{b}^{\dagger} + \hat{b}) | \Phi_j^b,m\rangle$

In [26]:
n_s = len(singlesDets)
g_vec = np.zeros((1,n_s))
for i in range(n_s):
    g_vec[0,i] = g_generator.calcMatrixElement(refDet, singlesDets[i])
print(g_vec)
g_dag = np.conj(g_vec).T
print(g_dag)

[[0.01608 0.01608]]
[[0.01608]
 [0.01608]]


Now generate ${\bf \Omega}$ 

In [27]:
Omega_matrix = np.identity(n_s) * omega_val
print(Omega_matrix)

[[0.17456 0.     ]
 [0.      0.17456]]


Now build the entire matrix

In [32]:
# define the offsets
R0_offset = 0
S0_offset = 1
R1_offset = n_s + 1
S1_offset = n_s + 2

H_CISS_PF = np.zeros((n_s * 2 + 2, n_s * 2 + 2))
H_CISS_PF[R0_offset:S0_offset, S1_offset:] = g_vec
H_CISS_PF[S0_offset:R1_offset, R1_offset:S1_offset] = g_dag
H_CISS_PF[R1_offset:S1_offset, S0_offset:R1_offset] = g_vec
H_CISS_PF[S1_offset:,          R0_offset:S0_offset] = g_dag

H_CISS_PF[S0_offset:R1_offset, S0_offset:R1_offset] = A_D_matrix

H_CISS_PF[R1_offset, R1_offset] = omega_val

H_CISS_PF[S1_offset:, S1_offset:] = A_D_matrix + Omega_matrix

H_CISS_PF[S1_offset:,S0_offset:R1_offset] = G_matrix
H_CISS_PF[S0_offset:R1_offset, S1_offset:] = G_matrix


In [34]:
print('..finished generating Matrix in %.3f seconds.\n' % (time.time() - t))

print('Diagonalizing Hamiltonian Matrix...')

t = time.time()

e_fci, wavefunctions = np.linalg.eigh(H_CISS_PF)
print(e_fci)

..finished generating Matrix in 14.304 seconds.

Diagonalizing Hamiltonian Matrix...
[-1.27069 -1.09613 -0.87168 -0.69737  0.00074  0.17505]


In [39]:
print('..finished diagonalization in %.3f seconds.\n' % (time.time() - t))
for i in range(0,len(e_fci)):
    en = e_fci[i] + mol.nuclear_repulsion_energy() + dc - cqed_scf_e
    print(F'{en}\n')
fci_mol_e = e_fci[3] + mol.nuclear_repulsion_energy() + dc

print('# Determinants:     % 16d' % (len(singlesDets)))

print('SCF energy:         % 16.10f' % (scf_e))
print('FCI correlation:    % 16.10f' % (fci_mol_e - scf_e))
print('Total FCI energy:   % 16.10f' % (fci_mol_e))

..finished diagonalization in 318.148 seconds.

0.31771108719597885

0.4922703689800386

0.7167221575689043

0.8910342054744401

1.5891436243991675

1.763455672304703

# Determinants:                    2
SCF energy:            -1.0661086492
FCI correlation:        0.8979178770
Total FCI energy:      -0.1681907721


In [ ]:
assert np.isclose(0.71721636, 0.7172163598047324, 1e-7)

In [ ]:
print(dc)
print(Hamiltonian_matrix)

In [ ]:
print(H)

In [ ]:
print(MO)

In [ ]:
# Create instance of MintsHelper class
#mints = psi4.core.MintsHelper(wfn.basisset())
# 2 electron integrals in CQED-RHF basis
Ca = wfn.Ca()
#MO_spin = np.asarray(mints.mo_spin_eri(Ca, Ca))

MO_spat = np.asarray(mints.mo_eri(Ca, Ca, Ca, Ca))


In [ ]:
nso = 2 * nmo
MO_spin = np.zeros((nso, nso, nso, nso))
for i in range(nso):
    for j in range(nso):
        for k in range(nso):
            for l in range(nso):
                MO_spin[i, j, k, l] = map_spatial_to_spin(MO_spat, i, j, k, l)
                
                
    
    

In [ ]:
assert np.allclose(MO_spin, MO)